In [ ]:
import os
import json
from typing import List, Dict, Any, Optional
from typing_extensions import TypedDict

from langchain_google_genai import ChatGoogleGenerativeAI
from tavily import TavilyClient

In [2]:
GEMINI_KEY = os.getenv("GEMINI_API_KEY")
TAVILY_KEY = os.getenv("TAVILY_API_KEY")

In [ ]:
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=GEMINI_KEY, temperature=0.2)
tavily_client = TavilyClient(api_key=TAVILY_KEY)


In [ ]:
class Question(TypedDict):
    id: str
    type: str  
    title: str
    prompt: str
    difficulty: str
    time_limit_seconds: Optional[int]

class AnswerRecord(TypedDict):
    qid: str
    answer_text: str
    score: float
    strengths: List[str]
    weaknesses: List[str]
    suggested_resources: List[Dict[str, str]]

In [7]:
def extract_json_from_text(text: str) -> Dict[str, Any]:
    """Try to extract the first JSON object from LLM output robustly."""
    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in LLM response.")
    sub = text[start:end+1]
    return json.loads(sub)

In [9]:

def tavily_search(query: str, max_results: int = 5) -> List[Dict[str, str]]:
    """Return list of {title, url} from Tavily. Returns empty list if Tavily not configured."""
    if tavily_client is None:
        return []
    res = tavily_client.search(query=query, max_results=max_results)
    items = res.get("results", []) if isinstance(res, dict) else []
    return [{"title": r.get("title", ""), "url": r.get("url", "")} for r in items[:max_results]]


In [10]:
def generate_questions(role: str, level: str, user_skills: List[str], n_questions: int = 6) -> List[Question]:
    """
    Ask Gemini to generate N tailored questions for role+level.
    Returns a list of question dicts (Question TypedDict).
    """
    skills_text = ", ".join(user_skills)
    prompt = f"""
You are an interview-question generator. Produce a JSON object with key "questions"
which is a list of {n_questions} question objects tailored for role "{role}" and level "{level}".
Each question object must have keys:
  - id (string)
  - type (behavioral|technical|coding|system-design)
  - title
  - prompt (what the interviewer will ask)
  - difficulty (easy|medium|hard)
  - time_limit_seconds (integer or null)

Make technical questions focus on the user's weaker skills if provided: {skills_text}.
Return ONLY a single JSON object. No extra commentary.
"""
    resp = gemini_llm.invoke([{"role": "user", "content": prompt}])
    out = resp.content
    try:
        data = extract_json_from_text(out)
        questions = data.get("questions", [])
        return questions
    except Exception as e:
        raise RuntimeError(f"Failed to parse questions from LLM output: {e}\nRaw output:\n{out}")


In [11]:

def create_suggested_resources(skills: List[str]) -> List[Dict[str, str]]:
    """For each skill, fetch top Tavily resources and flatten into a list (title, url)."""
    resources = []
    for s in skills:
        hits = tavily_search(f"best free resources to learn {s}", max_results=3)
        if hits:
            for h in hits:
                resources.append({"title": f"{s}: {h['title']}", "url": h["url"]})
        else:
            # fallback placeholder
            resources.append({"title": f"Search resources for {s}", "url": f"https://www.google.com/search?q=learn+{s.replace(' ', '+')}"})
    return resources

In [ ]:

def run_text_mock_interview(role: str, level: str, user_skills: List[str]) -> Dict[str, Any]:
    """
    CLI flow: generate questions, present them, collect typed answers, grade with Gemini, return a session summary.
    """
    print(f"Preparing {role} ({level}) mock interview with skills: {user_skills}...\n")
    questions = generate_questions(role, level, user_skills, n_questions=6)
    answers: List[AnswerRecord] = []

    for i, q in enumerate(questions, start=1):
        print(f"Q{i}. ({q.get('type')}) {q.get('title')}")
        print(q.get("prompt"))
        user_ans = input("\nYour answer (type + Enter):\n")
        grade = grade_answer_with_gemini(q, user_ans)
        suggested_skills = grade.get("suggested_skills_to_improve", [])
        suggested_resources = create_suggested_resources(suggested_skills)
        record: AnswerRecord = {
            "qid": q.get("id"),
            "answer_text": user_ans,
            "score": float(grade.get("score", 0.0)),
            "strengths": grade.get("strengths", []),
            "weaknesses": grade.get("weaknesses", []),
            "suggested_resources": suggested_resources,
        }
        answers.append(record)
        print(f"\n--- Feedback (score {record['score']:.1f}) ---")
        for s in record["strengths"]:
            print(f" + {s}")
        for w in record["weaknesses"]:
            print(f" - {w}")
        print(" Suggested resources:")
        for r in record["suggested_resources"][:5]:
            print(f"   * {r['title']} - {r['url']}")
        print("\n" + ("="*60) + "\n")


    avg_score = sum(a["score"] for a in answers) / len(answers) if answers else 0.0

    weak_skills = []
    for a in answers:
        for res in a["suggested_resources"]:

            t = res["title"]
            if ":" in t:
                skill_name = t.split(":", 1)[0].strip()
                weak_skills.append(skill_name)

    weak_skills = list(dict.fromkeys(weak_skills))[:6]

    roadmap = "📅 Suggested Short Roadmap:\n"
    day = 1
    for s in weak_skills:
        roadmap += f"\nDay {day}-{day+3}: Focus on {s}\n"
        hits = tavily_search(f"best free resources to learn {s}", max_results=4)
        for idx, h in enumerate(hits, start=1):
            roadmap += f"  {idx}. {h['title']} ({h['url']})\n"
        day += 4

    session = {
        "role": role,
        "level": level,
        "questions": questions,
        "answers": answers,
        "avg_score": avg_score,
        "roadmap": roadmap,
    }
    return session


In [13]:
def _cli():
    print("=== Mock Interview CLI (MVP) ===")
    role = input("Target role (e.g., Data Scientist): ").strip()
    level = input("Level (junior/mid/senior): ").strip() or "junior"
    skills = input("Your skills (comma separated): ").strip()
    user_skills = [s.strip().lower() for s in skills.split(",") if s.strip()]
    session = run_text_mock_interview(role, level, user_skills)
    print("\n\n=== SESSION SUMMARY ===")
    print(f"Average score: {session['avg_score']:.1f}")
    print(session["roadmap"])


if __name__ == "__main__":
    _cli()


=== Mock Interview CLI (MVP) ===
Preparing AI/ML engineer (junior) mock interview with skills: ['machine learning', 'deep learning', 'docker', 'sql']...

Q1. (behavioral) Teamwork and Collaboration
Describe a time you worked on a project with a team and faced a disagreement. How did you handle it, and what was the outcome?


NameError: name 'grade_answer_with_gemini' is not defined